# Imports

In [11]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly ins

In [12]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [13]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [14]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización de los RDD

In [15]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

In [16]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

apps_df = sqlContext.read.csv('googleplaystore.csv',
                              header=True,
                              inferSchema=True,
                              mode='DROPMALFORMED') \
                              .select("App", "Rating", "Type")
apps_rdd = apps_df.rdd

# Pregunta #15
[*ir al notebook general*](https://colab.research.google.com/drive/1wWj79hKJTtaauQYJndtlhfQ8KEiuKe8Q?ouid=101857797731019568886&usp=drive_link)

> *Calcular el promedio de rating por tipo de App*

**Respuesta:** 4.17 para las gratis y 4.26 para las pagas

**Notas:**
- Como hay *una sola* línea malformada que no me sacó spark al cargar el csv, decidí dropearla a mano (por eso el `x[0] != '0'`)
- El primer map y reduceByKey me quitan las apps duplicadas. Hago básicamente lo mismo (map a dupla nombre - campos que quiero y reduceByKey de agarrar uno de los 2) en el resto de los ejercicios que usan el dataset de apps. No tengo ningún criterio para seleccionar a uno de los duplicados, así que agarro uno cualquiera
- La función `to_float()` es un poco bruta para mi gusto pero es sencilla y hace exactamente lo que quiero (pasarme las cosas a float y volar todo lo que no sea un float)

In [17]:
def to_float(str):
  try:
    return float(str)
  except:
    return math.nan

apps_rdd \
  .map(lambda x: (x[0], (x[1], x[2]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[1][1], (to_float(x[1][0]), 1))) \
  .filter(lambda x: not math.isnan(x[1][0]) and x[0] != '0') \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .take(2)

[('Free', 4.166337285902508), ('Paid', 4.259602649006619)]